In [31]:
library(tidyr)
library(dplyr)
library(utils)


ERROR: Error in library(here): there is no package called ‘here’


ERROR: Error in here(): could not find function "here"


In [45]:
#Generating File Paths
gen.atac.path <- function(pid, data.path, summary){
    
    summ = read.csv(paste(data.path, summary, sep=""))
    
    p.summ = summ[summ$Patient == pid,]
    
    atac.date <- p.summ$date_ATAC
    atac.id <- p.summ$sampleid_ATAC
    atac.tissue <- p.summ$Tissue
    
    atac.path <- paste(data.path,
                       "scATAC-seq_",
                       atac.tissue, ".",
                       atac.date,
                       "/",
                       atac.id,
                       "/outs/",
                       sep="")
    return(atac.path)
    
}

gen.rna.path <- function(pid, data.path, summary){
    
    summ = read.csv(paste(data.path, summary, sep=""))
    
    p.summ = summ[summ$Patient == pid,]
    
    atac.date <- p.summ$date_RNA
    atac.id <- p.summ$sampleid_RNA
    atac.tissue <- p.summ$Tissue
    
    atac.path <- paste(data.path,
                       "scRNA-seq_",
                       atac.tissue, ".",
                       atac.date,
                       "/",
                       atac.id,
                       "/outs/",
                       sep="")
    return(atac.path)
    
}


base.path <- "/home/cyogodzi/projects/franco/"



summ <- "Single-cell_sequencing.summary.09.14.20.csv"
patient.id <- "43E7BL"

data.path <- paste(base.path, "francoData/", sep="")
atac.path <- gen.atac.path(patient.id, data.path, summ)
atac.peaks <- paste(atac.path, "peak_annotation.tsv", sep="")

rna.path <- gen.rna.path(patient.id, data.path, summ)

In [49]:
paste(rna.path, "possorted_genome_bam.bam", sep="/")

[1] "/home/cyogodzi/projects/franco/francoData/scRNA-seq_Breast.08.10.2020/43E7BL-RNA/outs//possorted_genome_bam.bam"

In [42]:
atac.peaks

[1] "/home/cyogodzi/projects/franco/francoData/scATAC-seq_Breast.08.07.2020/43E7BL-ATAC/outs/peak_annotation.tsv"

In [3]:
#read peak annotation file
peak.annot <- read.delim(atac.peaks)
head(peak.annot)

,peak,gene,distance,peak_type
,<chr>,<chr>,<chr>,<chr>
1,chr1_191364_191721,OR4F5,122310,distal
2,chr1_267837_268176,OR4F29,183521,distal
3,chr1_629771_630127,OR4F16,56546,distal
4,chr1_633798_634264,OR4F16,52409,distal
5,chr1_778190_780050,OR4F16,-91518,distal
6,chr1_816876_818224,SAMD11,-107514,distal


In [25]:
noncoding <- peak.annot %>% 
             group_by(peak_type) %>%
             filter(peak_type == "distal" || peak_type == "intergenic")

In [30]:
bed <- tidyr::separate(noncoding, col=peak, into = c("chrom", "start", "end"), sep="_")

In [50]:
write.table(bed[,1:3], paste("out", patient.id,"noncoding-beds","noncoding_peaks_43E7BL.bed", sep="/"), sep="\t", quote=F, row.names=F, col.names=F)

[1] "out/43E7BL/noncoding_peaks_43EB7L.bed"